In [9]:
import os
# Find the latest version of spark 3.x from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION'] = spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

# Import required PySpark functions
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, round
import time

# Create a SparkSession
spark = SparkSession.builder.appName("HomeSales").getOrCreate()

# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")

# 3. What is the average price for a four-bedroom house sold per year, rounded to two decimal places?
query_1 = """
SELECT date_format(date, 'yyyy') as year,
       ROUND(AVG(price), 2) as avg_price
FROM home_sales
WHERE bedrooms = 4
GROUP BY year
ORDER BY year
"""
spark.sql(query_1).show()

# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
query_2 = """
SELECT date_built,
       ROUND(AVG(price), 2) as avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY date_built
ORDER BY date_built
"""
spark.sql(query_2).show()

# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors, and are greater than or equal to 2,000 square feet,
# rounded to two decimal places?
query_3 = """
SELECT date_built,
       ROUND(AVG(price), 2) as avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
GROUP BY date_built
ORDER BY date_built
"""
spark.sql(query_3).show()

# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Determine the runtime for this query.
start_time = time.time()

query_4 = """
SELECT view,
       ROUND(AVG(price), 2) as avg_price
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY avg_price DESC
"""
spark.sql(query_4).show()

print("--- %s seconds ---" % (time.time() - start_time))

# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales")

# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.
start_time = time.time()

spark.sql(query_4).show()

print("--- Cached query runtime: %s seconds ---" % (time.time() - start_time))

# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.mode("overwrite").partitionBy("date_built").parquet("home_sales_parquet")

# 11. Read the parquet formatted data.
parquet_df = spark.read.parquet("home_sales_parquet")

# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView("parquet_home_sales")

# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.
start_time = time.time()

spark.sql(query_4).show()

print("--- Parquet query runtime: %s seconds ---" % (time.time() - start_time))

# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

# 15. Check if the home_sales is no longer cached.
spark.catalog.isCached("home_sales")

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
tar: spark-3

False